In [ ]:
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import regularizers  
import re

In [ ]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
from kaggle_datasets import KaggleDatasets
GCS_DS_PATH = KaggleDatasets().get_gcs_path()
!gsutil ls $GCS_DS_PATH

In [ ]:
Train_path=GCS_DS_PATH+'/tfrecords-jpeg-512x512/train/'
val_path=GCS_DS_PATH+'/tfrecords-jpeg-512x512/val/'
test_path=GCS_DS_PATH+'/tfrecords-jpeg-512x512/test/'
train_files=tf.io.gfile.glob(Train_path +'*.tfrec')
val_files=tf.io.gfile.glob(val_path +'*.tfrec')
test_files=tf.io.gfile.glob(test_path +'*.tfrec')

In [ ]:
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']   

In [ ]:
IMAGE_SIZE        = [512, 512] 
HEIGHT            = IMAGE_SIZE[0]
WIDTH             = IMAGE_SIZE[1]
EPOCHS            = 20
BATCH_SIZE        = 16 * strategy.num_replicas_in_sync
NUM_TRAIN_IMAGES  = 12753
NUM_VAL_IMAGES    = 3712
NUM_TEST_IMAGES   = 7382
STEPS_PER_EPOCH   = NUM_TRAIN_IMAGES // BATCH_SIZE
AUTO              = tf.data.experimental.AUTOTUNE

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

In [ ]:
def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

In [ ]:
def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),    # shape [] means single element
        # class is missing, to be predicted flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of (image, idnum) pairs

In [ ]:
def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    
    # automatically interleaves reads from multiple file
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) 
    
    # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.with_options(ignore_order) 
    
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    dataset = dataset.map(read_labeled_tfrecord if labeled 
                          else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    return dataset

In [ ]:
def get_validation_dataset(filenames):
    dataset = load_dataset(filenames,labeled=True, ordered=False)
    dataset = dataset.cache()
    dataset = dataset.shuffle(buffer_size=1920)
    dataset = dataset.batch(BATCH_SIZE)
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

def get_test_dataset(filenames, ordered=True):  # order matters to submit predictions to Kaggle
    dataset = load_dataset(filenames, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

In [ ]:
# image augmentation, one image at a time                                
def data_augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, 0.90, 0.99)
    image = tf.image.random_brightness(image, 0.1) 
    image = tf.image.random_saturation(image, 0.8, 0.9)

    # Pad the image with a black, 90-pixel border
    image = tf.image.resize_with_crop_or_pad(image, HEIGHT + 180, WIDTH + 180)
    # Randomly crop to original size from the padded image
    image = tf.image.random_crop(image, size=[*IMAGE_SIZE,3])
     
    return image, label 

# get training datatset with augmentation option
def get_training_dataset(filenames, augmentation=False):
    dataset = load_dataset(filenames, labeled=True, ordered=False)
    if augmentation: # map the data_augment function to the dataset during training
        dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(buffer_size=1920)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)  # prefetch next batch while training
    return dataset

In [ ]:
np.set_printoptions(threshold=15, linewidth=80)

def batch_to_numpy_images_and_labels(data):
    images, labels = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    if numpy_labels.dtype == object: # binary string in this case, these are image ID strings
        numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is the case for test data)
    return numpy_images, numpy_labels

def show_images(databatch, row=6, col=8):
    FIGSIZE = col*2
    plt.figure(figsize=(FIGSIZE,FIGSIZE/col*row))
    images, num_labl = batch_to_numpy_images_and_labels(databatch)
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.title(num_labl[j])
        plt.imshow(images[j,])
    plt.show()

In [ ]:
def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = count_data_items(train_files)
NUM_VALIDATION_IMAGES = count_data_items(val_files)
NUM_TEST_IMAGES = count_data_items(test_files)

In [ ]:
# get original training_dataset WITHOUT data augmentation
ori_train_set = get_training_dataset(train_files, augmentation=False)
ori_image_batch = (next(iter(ori_train_set.unbatch().batch(16)))) # get a batch for 
images, _ = batch_to_numpy_images_and_labels(ori_image_batch)

# function to show image with random data augmentation
def show_aug(image):
    plt.figure(figsize=(16,2))
    
    plt.subplot(1,7,1)
    plt.title('no augmentation')
    plt.axis('off')
    plt.imshow(image)
    
    plt.subplot(1,7,3)
    plt.title('resize & rdm crop')
    plt.axis('off')    
    # Pad the image with a black, 90-pixel border
    image1 = tf.image.resize_with_crop_or_pad(image, HEIGHT + 180, WIDTH + 180)
    # Randomly crop to original size from the padded image
    image1 = tf.image.random_crop(image1, size=[*IMAGE_SIZE,3])
    plt.imshow(image1)
    
    plt.subplot(1,7,4)
    plt.title('rdm flip L/R')
    plt.axis('off')    
    plt.imshow(tf.image.random_flip_left_right(image))       # augmented with random flip
    
    plt.subplot(1,7,5)
    plt.title('rdm contrast')
    plt.axis('off')
    plt.imshow(tf.image.random_contrast(image, 0.90, 0.99))  # augmented with contrast
    
    plt.subplot(1,7,6)
    plt.title('rdm brightness')
    plt.axis('off')
    plt.imshow(tf.image.random_brightness(image, 0.15))       # augmented with brightness
    
    plt.subplot(1,7,7)
    plt.title('rdm saturation')
    plt.axis('off')
    plt.imshow(tf.image.random_saturation(image, 0.75, 0.9))  # augmented with saturation
     
    # any random combinations of the above augmenations, if any
    plt.subplot(1,7,2)
    plt.title('rdm aug combo')
    plt.axis('off')    
    image = data_augment(image, None)
    plt.imshow(image[0])  
    plt.show()

# show images
print('Training Dataset')
print('Sample Images: Original versus w/ Random Augmentation')
for im in images:
    show_aug(im)

In [ ]:
ds_train = get_training_dataset(train_files)
ds_valid = get_validation_dataset(val_files)
ds_test = get_test_dataset(test_files)

print("Training:", ds_train)
print ("Validation:", ds_valid)
print("Test:", ds_test)

In [ ]:
print("Training data shapes:")
for image, label in ds_train.take(3):
    print(image.numpy().shape, label.numpy().shape)
print("Training data label examples:", label.numpy())

In [ ]:
image_input, label_val = next(iter(ds_train))

In [ ]:
fig = plt.figure(figsize=(10,5)) # define plot area
ax = fig.gca() # define axis    
counts = pd.DataFrame(label_val.numpy()).value_counts() # find the counts for each unique category
counts.plot.bar(ax = ax, color = 'blue') # Use the plot.bar method on the counts data frame

ax.set_xlabel('Labels') # Set text for the x axis
ax.set_ylabel('Number of occurences')# Set text for y axis
plt.show()

In [ ]:
pd.DataFrame(label_val.numpy()).value_counts()

In [ ]:
import matplotlib.pyplot as plt

from PIL import Image
import numpy as np

for i in range(0,5):
    
    im = image_input[i]
  

   # Create figure and axes
    fig,ax = plt.subplots(1)

    # Display the image
    ax.imshow(im)
    plt.title(CLASSES[label_val[i].numpy()])
    plt.show()

In [ ]:
index=4
name=CLASSES[index]
print(name)
for i in range(len(image_input)):
    
    if label_val[i].numpy()==index:
        
        im = image_input[i]


       # Create figure and axes
        fig,ax = plt.subplots(1)

        # Display the image
        ax.imshow(im)
        
        plt.show()

In [ ]:
index=53
name=CLASSES[index]
print(name)
for i in range(len(image_input)):
    
    if label_val[i].numpy()==index:
        
        im = image_input[i]


       # Create figure and axes
        fig,ax = plt.subplots(1)

        # Display the image
        ax.imshow(im)
        
        plt.show()

In [ ]:
index=102
name=CLASSES[index]
print(name)
for i in range(len(image_input)):
    
    if label_val[i].numpy()==index:
        
        im = image_input[i]


       # Create figure and axes
        fig,ax = plt.subplots(1)

        # Display the image
        ax.imshow(im)
        
        plt.show()

In [ ]:
index=72
name=CLASSES[index]
print(name)
for i in range(len(image_input)):
    
    if label_val[i].numpy()==index:
        
        im = image_input[i]


       # Create figure and axes
        fig,ax = plt.subplots(1)

        # Display the image
        ax.imshow(im)
        
        plt.show()

In [ ]:
index=67
name=CLASSES[index]
print(name)
for i in range(len(image_input)):
    
    if label_val[i].numpy()==index:
        
        im = image_input[i]


       # Create figure and axes
        fig,ax = plt.subplots(1)

        # Display the image
        ax.imshow(im)
        
        plt.show()

In [ ]:
label_great_2=pd.DataFrame(label_val.numpy()).value_counts()[pd.DataFrame(label_val.numpy()).value_counts()>=2]

In [ ]:
# Learning Rate Schedule for Fine Tuning #
def exponential_lr(epoch,
                   start_lr = 0.00001, min_lr = 0.00001, max_lr = 0.00005,
                   rampup_epochs = 5, sustain_epochs = 0,
                   exp_decay = 0.8):

    def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
        # linear increase from start to rampup_epochs
        if epoch < rampup_epochs:
            lr = ((max_lr - start_lr) /
                  rampup_epochs * epoch + start_lr)
        # constant max_lr during sustain_epochs
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr
        # exponential decay towards min_lr
        else:
            lr = ((max_lr - min_lr) *
                  exp_decay**(epoch - rampup_epochs - sustain_epochs) +
                  min_lr)
        return lr
    return lr(epoch,
              start_lr,
              min_lr,
              max_lr,
              rampup_epochs,
              sustain_epochs,
              exp_decay)

lr_callback = tf.keras.callbacks.LearningRateScheduler(exponential_lr, verbose=True)

rng = [i for i in range(EPOCHS)]
y = [exponential_lr(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
with strategy.scope():
    pretrained_model = tf.keras.applications.DenseNet201(
        weights='imagenet',
        include_top=False ,
        input_shape=[*IMAGE_SIZE, 3]
    )
    pretrained_model.trainable = False
    
    model = tf.keras.Sequential([
        # To a base pretrained on ImageNet to extract features from images...
        pretrained_model,
        # ... attach a new head to act as a classifier.
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(512,kernel_regularizer=regularizers.l2(0.001)),
        
        tf.keras.layers.Dense(256,kernel_regularizer=regularizers.l2(0.001)),
        
        tf.keras.layers.Dense(128,kernel_regularizer=regularizers.l2(0.001)),
        
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss = 'sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy'],
    )

model.summary()

In [ ]:
# Define the batch size. This will be 16 with TPU off and 128 with TPU on
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

# Define training epochs for committing/submitting. (TPU on)

STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[lr_callback]
)

In [ ]:
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

In [ ]:
display_training_curves(
    history.history['loss'],
    history.history['val_loss'],
    'loss',
    211,
)
display_training_curves(
    history.history['sparse_categorical_accuracy'],
    history.history['val_sparse_categorical_accuracy'],
    'accuracy',
    212,
)

In [ ]:
#model.save('model_vgc.h5')

In [ ]:
"""# With pretrained model: DenseNet201
with strategy.scope():    
    pretrain_model = tf.keras.applications.InceptionV3(
        weights='imagenet', 
        include_top=False ,
        input_shape=[*IMAGE_SIZE, 3]
    )
    pretrain_model.trainable = False # transfer learning
    model_incep = tf.keras.Sequential([
        pretrain_model,
#        tf.keras.layers.experimental.preprocessing.RandomRotation(0.15),
#        tf.keras.layers.experimental.preprocessing.RandomZoom(0.75),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.3),  
        tf.keras.layers.Dense(len(CLASSES), kernel_regularizer=regularizers.l2(0.001), 
                              activation='softmax')
    ])
    
    model_incep.compile(
        optimizer='adam',
        loss = 'sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy'],
    )

model_incep.summary()"""

In [ ]:
"""
historical = model_incep.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[lr_callback]
)"""

In [ ]:
"""display_training_curves(
    historical.history['loss'],
    historical.history['val_loss'],
    'loss',
    211,
)
display_training_curves(
    historical.history['sparse_categorical_accuracy'],
    historical.history['val_sparse_categorical_accuracy'],
    'accuracy',
    212,
)"""

In [ ]:
"""# With pretrained model: DenseNet201
with strategy.scope():    
    pretrain_model_Inceptio = tf.keras.applications.InceptionResNetV2(
        weights='imagenet', 
        include_top=False ,
        input_shape=[*IMAGE_SIZE, 3]
    )
    pretrain_model_Inceptio.trainable = False # transfer learning
    model_Inceptio = tf.keras.Sequential([
         pretrain_model_Inceptio,
#        tf.keras.layers.experimental.preprocessing.RandomRotation(0.15),
#        tf.keras.layers.experimental.preprocessing.RandomZoom(0.75),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.3),  
        tf.keras.layers.Dense(len(CLASSES), kernel_regularizer=regularizers.l2(0.001), 
                              activation='softmax')
    ])
    
    model_Inceptio.compile(
        optimizer='adam',
        loss = 'sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy'],
    )

model_Inceptio.summary()"""

In [ ]:
"""
historical_inc = model_Inceptio.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[lr_callback]
)"""

In [ ]:
"""display_training_curves(
    historical_inc.history['loss'],
    historical.history['val_loss'],
    'loss',
    211,
)
display_training_curves(
    historical_inc.history['sparse_categorical_accuracy'],
    historical.history['val_sparse_categorical_accuracy'],
    'accuracy',
    212,
)"""

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

def display_confusion_matrix(cmat, score, precision, recall):
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Reds')
    ax.set_xticks(range(len(CLASSES)))
    ax.set_xticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(len(CLASSES)))
    ax.set_yticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.3f} '.format(recall)
    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()
    
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

In [ ]:
cmdataset = get_validation_dataset(val_files)
images_ds = cmdataset.map(lambda image, label: image)
labels_ds = cmdataset.map(lambda image, label: label).unbatch()

cm_correct_labels = next(iter(labels_ds.batch(NUM_VALIDATION_IMAGES))).numpy()
cm_probabilities = model.predict(images_ds)

cm_predictions = np.argmax(cm_probabilities, axis=-1)

labels = range(len(CLASSES))
cmat = confusion_matrix(
    cm_correct_labels,
    cm_predictions,
    labels=labels,
)
cmat = (cmat.T / cmat.sum(axis=1)).T # normalize

In [ ]:
score = f1_score(
    cm_correct_labels,
    cm_predictions,
    labels=labels,
    average='macro',
)
precision = precision_score(
    cm_correct_labels,
    cm_predictions,
    labels=labels,
    average='macro',
)
recall = recall_score(
    cm_correct_labels,
    cm_predictions,
    labels=labels,
    average='macro',
)
display_confusion_matrix(cmat, score, precision, recall)

In [ ]:
test_images_ds = ds_test.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

In [ ]:
"""

print('Computing predictions...')
test_images_ds = ds_test.map(lambda image, idnum: image)
probabilities_1 = model.predict(test_images_ds)
probabilities_2 = model_incep.predict(test_images_ds)
probabilities_3 = model_Inceptio.predict(test_images_ds)
ensemble_11=probabilities_1 *.5 +  probabilities_2*.5
ensemble_12=probabilities_1 *.25 +  probabilities_2*.75
ensemble_13=probabilities_1 *.75 +  probabilities_2*.25

ensemble_21=probabilities_1 *.5 +  probabilities_3*.5
ensemble_22=probabilities_1 *.25 +  probabilities_3*.75
ensemble_23=probabilities_1 *.75 +  probabilities_3*.25

ensemble_31=probabilities_2 *.5 +  probabilities_3*.5
ensemble_32=probabilities_2 *.25 +  probabilities_3*.75
ensemble_33=probabilities_2 *.75 +  probabilities_3*.25"""

In [ ]:
"""predictions11 = np.argmax(ensemble_11, axis=-1)
print(predictions)"""

In [ ]:
"""predictions12 = np.argmax(ensemble_12, axis=-1)
print(predictions12)"""

In [ ]:
"""predictions13 = np.argmax(ensemble_13, axis=-1)
print(predictions13)"""

In [ ]:
"""predictions21 = np.argmax(ensemble_21, axis=-1)
predictions22 = np.argmax(ensemble_22, axis=-1)
predictions23 = np.argmax(ensemble_23, axis=-1)

predictions31 = np.argmax(ensemble_31, axis=-1)
predictions32 = np.argmax(ensemble_32, axis=-1)
predictions33 = np.argmax(ensemble_33, axis=-1)

"""


In [ ]:
print('Generating submission.csv file...')

# Get image ids from test set and convert to unicode
test_ids_ds = ds_test.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

# Write the submission file
np.savetxt(
    'submission.csv',
    np.rec.fromarrays([test_ids, predictions]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)

# Look at the first few predictions
!head submission.csv

In [ ]:
"""print('Generating submission.csv file...')

# Get image ids from test set and convert to unicode
test_ids_ds = ds_test.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

# Write the submission file
np.savetxt(
    'submission2.csv',
    np.rec.fromarrays([test_ids, predictions12]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)

# Write the submission file
np.savetxt(
    'submission3.csv',
    np.rec.fromarrays([test_ids, predictions13]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)

# Write the submission file
np.savetxt(
    'submission4.csv',
    np.rec.fromarrays([test_ids, predictions21]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)

# Write the submission file
np.savetxt(
    'submission5.csv',
    np.rec.fromarrays([test_ids, predictions22]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)

# Write the submission file
np.savetxt(
    'submission6.csv',
    np.rec.fromarrays([test_ids, predictions23]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)

# Write the submission file
np.savetxt(
    'submission7.csv',
    np.rec.fromarrays([test_ids, predictions31]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)

# Write the submission file
np.savetxt(
    'submission8.csv',
    np.rec.fromarrays([test_ids, predictions32]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)

# Write the submission file
np.savetxt(
    'submission9.csv',
    np.rec.fromarrays([test_ids, predictions33]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)

# Look at the first few predictions
!head submission2.csv"""